# Taller de data quality

Juan Sebastian Alvarez Eraso

Código: 201822427

---

MINE-4101: Applied Data Science  
Universidad de los Andes  
  
Dataset: Homicides Colombia  
Source: [datos.gov.co](datos.gov.co)
  
Last update: September, 2022

# Configuración inicial

In [44]:
!pip install pylev

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
# Importing required libraries
import re
from random import randint
from datetime import datetime

import numpy as np
import pandas as pd

import pylev

In [46]:
# Parameter for showing all columns when printing a dataframe
pd.set_option('display.max_columns', None)

# Cargar dataset

In [47]:
# Loading data
homicides_df = pd.read_csv("https://raw.githubusercontent.com/juanalvarez123/MINE-4101-taller-data-quality/main/Dataset/homicides.csv", sep = ',')

In [48]:
# Printing the dataset dimensions
homicides_df.shape

(12400, 22)

In [49]:
# Printing column data types
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [50]:
homicides_df.head(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,01/01/2021 12:00:00 AM,ANTIOQUIA,AMAGÁ,Jueves,6:00,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,01/01/2021 12:00:00 AM,ANTIOQUIA,BARBOSA,Jueves,9:00,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,01/01/2021 12:00:00 AM,ANTIOQUIA,EL BAGRE,Jueves,19:00,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,01/01/2021 12:00:00 AM,ANTIOQUIA,JARDÍN,Jueves,11:20,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Juees,15:00,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co


# Manipulando las columnas "FECHA" y "HORA"

In [51]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [52]:
# Applying the validation to all values in column
# homicides_df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR INTENTIONALLY

La ejecución del bloque anterior lanza un error:

```bash
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-9-4adcb6515e66> in <module>
      1 # Applying the validation to all values in column
----> 2 homicides_df["FECHA"].apply(dateparse)
      3 
      4 # IT IS EXPECTED TO HAVE AN ERROR INTENTIONALLY

6 frames
/usr/lib/python3.7/_strptime.py in _strptime(data_string, format)
    357     if not found:
    358         raise ValueError("time data %r does not match format %r" %
--> 359                          (data_string, format))
    360     if len(data_string) != found.end():
    361         raise ValueError("unconverted data remains: %s" %

ValueError: time data 'TOTAL' does not match format '%m/%d/%Y %H:%M:%S %p'
```

In [53]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [54]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
12399,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


La última fila está causando el error

In [55]:
# Deleting a row by its index
homicides_df.drop(12399, inplace = True)

In [56]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [57]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

12    12399
Name: FECHA, dtype: int64

Todas las horas de la columna "FECHA" están seteadas a las 12:00:00 por eso el resultado de las "horas" dentro de la columna "FECHA" es siempre 12

In [58]:
# The homicide hour is available in a different column!
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11]) + homicides_df["HORA"]

In [59]:
del homicides_df["HORA"]

In [60]:
homicides_df["FECHA"].head(5)

0     2021-01-01 6:00
1     2021-01-01 9:00
2    2021-01-01 19:00
3    2021-01-01 11:20
4    2021-01-01 15:00
Name: FECHA, dtype: object

In [61]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [62]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [63]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [64]:
homicides_df.head(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 06:00:00,ANTIOQUIA,AMAGÁ,Jueves,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co


# Ajustando los departamentos

In [65]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS',
                    'CAQUETÁ', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA',
                    'META', 'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO',
                    'PUTUMAYO', 'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA',
                    'VALLE', 'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO',
                    'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA', 'GUAINÍA', 'VICHADA',
                    'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [66]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary</span>

<span style="color:red">Hint: You can use the replace() pandas function</span>

In [67]:
# Busco las filas que tengan como departamentos a 'SAN ANDRÉS' y 'N. DE SANTANDER'
# para colocar sus valores correctos
homicides_df.loc[homicides_df['DEPARTAMENTO'] == 'SAN ANDRÉS', 'DEPARTAMENTO'] = 'SAN ANDRÉS Y PROVIDENCIA'
homicides_df.loc[homicides_df['DEPARTAMENTO'] == 'N. DE SANTANDER'] = 'NORTE DE SANTANDER'

# Valido nuevamente cuantos departamentos están por fuera de "departments_list"
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array([], dtype=object)

Ya no hay departamentos por fuera de "departments_list"

# Duplicados y errores en la columna "CÉDULA"

In [68]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep = False)]

In [69]:
duplicates_by_cedula.shape

(1695, 21)

In [70]:
# Showing some examples
duplicates_by_cedula.sort_values("CÉDULA", ascending = True).head(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4041,2021-05-03 23:30:00,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com
3810,2021-04-26 03:00:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 05:30:00,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co
10796,2021-11-20 03:00:00,VALLE,CALI (CT),Viernes,POTRERO GRANDE E21,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1994.0,10-079,iknnoj8430@unidatos.edu.co
138,2021-01-02 06:20:00,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

In [71]:
homicides_df.drop_duplicates(subset = ['CÉDULA'], keep=False, inplace=True)

# Verifico ahora el tamaño del dataset, antes tenía 12400 registros, menos la
# última fila que fue eliminada (Por el error en el formateo de la fecha) y menos
# los 1695 duplicados en la cédula deberíamos tener ahora 10704 registros
homicides_df.shape

(10704, 21)

In [72]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [73]:
cedula_malformed.shape

(28, 21)

Hay 28 registros con la cédula malformada

In [74]:
cedula_malformed.head(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,680-21,oohghd8899@gmail.com
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,975-31,ohbqrk3631@unidatos.edu.co
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,348-66,pdkqur8407@unidatos.edu.co
3409,2021-04-13 23:20:00,CÓRDOBA,SAHAGÚN,Lunes,CORREGIMIENTO DE BAJO GRANDE,URBANA,BILLARES,CONTUNDENTES,A PIE,A PIE,49.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,23660000.0,1966.0,496-18,fdbbeo6751@unidatos.edu.co


<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

In [75]:
# Creo una función para arreglar las cédulas que están mal formadas
def fix_malformed_cedula(cedula):
  if not re.match("\d{2}-\d{3}", cedula):
    cedula = cedula.replace('-', '')
    cedula = cedula[:2] + '-' + cedula[2:]
  return cedula

# Aplico la función creada
homicides_df['CÉDULA'] = homicides_df['CÉDULA'].apply(fix_malformed_cedula)

# Verifico nuevamente la cantidad de cédulas malformadas
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]
cedula_malformed.shape

(0, 21)

Ya no hay cédulas malformadas

# Ajustando los correos malformados

<span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

In [76]:
# Busco la cantidad de correos que no cumplen con el patrón
correo_malformed = homicides_df.loc[homicides_df["CORREO"].apply(lambda x: (re.match("[a-zA-Z]+\d{4}@.*(.edu.co|.com)$", x) is None))]
correo_malformed.shape

(1983, 21)

Hay 1983 correos que no cumplen con la expresión regular

In [77]:
# Imprimo algunos ejemplos de correos malformados
correo_malformed.head(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
28,2021-01-01 15:00:00,CAQUETÁ,SAN JOSÉ DEL FRAGUA,Jueces,CENTRO,URBANA,DISCOTECAS,ARMA DE FUEGO,A PIE,A PIE,24.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18610000.0,1991.0,94-565,correo975@unidatos.edu.co
36,2021-01-01 08:00:00,CAUCA,PATÍA,Jueves,C/MIENTO PIEDRASENTADA,RURAL,CARRETERA,ARMA DE FUEGO,VEHICULO,A PIE,26.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19532000.0,1989.0,69-084,correo4714@colombia.gov.co
55,2021-01-01 01:30:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,ÁLVARO BERNAL SEGURA E-19,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,25.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1990.0,35-260,correo7285@colombia.gov.co
63,2021-01-01 17:53:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueces,LA ESTRELLA SECTOR LAGOS E-19,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,19.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1996.0,45-426,correo102@unidatos.edu.co
79,2021-01-01 16:00:00,NARIÑO,GUAITARILLA,Jueves,VEREDA AHUMADA CHIQUITA,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,26.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,52320000.0,1989.0,54-950,correo912@unidatos.edu.co


In [78]:
# Creo una función para detectar cuales son los correos que tienen el error y
# poder corregirlos
print_examples = 5

def fix_malformed_correo(correo):
  prev_correo = correo
  if not re.match("[a-zA-Z]+\d{4}@.*(.edu.co|.com)$", correo):
    if not re.match(".*(.edu.co|.com)$", correo):
      # Significa que tiene el error en el dominio
      correo = correo.replace('gov.co', 'com')

    if not re.match("[a-zA-Z]+\d{4}@.*", correo):
      # Significa que tiene el error en la cantidad de dígitos, no son 4
      arroba_index = correo.rindex("@")
      number = correo[6:arroba_index]
      number_just_4_digits = number[-4:]
      if len(number_just_4_digits) < 4:
        number_just_4_digits = number_just_4_digits.rjust(4, '0')
      correo = 'correo' + number_just_4_digits + correo[arroba_index:]
    
    global print_examples
    if print_examples > 0:
      print(f'Antes: {prev_correo}\tDespués: {correo}')

    print_examples = print_examples - 1
  return correo

# Aplico la función creada
print('Algunos ejemplos:')
homicides_df['CORREO'] = homicides_df['CORREO'].apply(fix_malformed_correo)

Algunos ejemplos:
Antes: correo975@unidatos.edu.co	Después: correo0975@unidatos.edu.co
Antes: correo4714@colombia.gov.co	Después: correo4714@colombia.com
Antes: correo7285@colombia.gov.co	Después: correo7285@colombia.com
Antes: correo102@unidatos.edu.co	Después: correo0102@unidatos.edu.co
Antes: correo912@unidatos.edu.co	Después: correo0912@unidatos.edu.co


In [79]:
# Verifico nuevamente la cantidad de correos malformados
correo_malformed = homicides_df.loc[homicides_df["CORREO"].apply(lambda x: (re.match("[a-zA-Z]+\d{4}@.*(.edu.co|.com)$", x) is None))]
correo_malformed.shape

(0, 21)

Ya no hay correos malformados

# Unificar los días de la semana

In [80]:
# Showing different values for "DIA" column
homicides_df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Ssbado', 'Domingo', 'Domungo', 'Doningo',
       'domungo', 'lune', 'Lunes', 'Luns', 'Lumes', 'kunes', 'Lnues',
       'Martes', 'Mates', 'Marte', 'mates', 'Miércoles', 'Miwrcoles',
       'Mircoles', 'Voernes', 'Sávado', 'domnigo', 'Maryes', 'Miercoles',
       'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'], dtype=object)

In [81]:
# Calculating the distance between two words using the Levenshtein method
pylev.levenshtein('sábado', 'sabaod')

3

In [82]:
pylev.levenshtein('sábado', 'viernes')

7

<span style="color:red">TODO: Create a function to fix the digitation errors for "DIA" column.</span>

In [83]:
# Creo una función que dependiendo del valor de la distancia devuelto por el
# método Levenshtein determine si cambia el día de la semana
def get_correct_day_of_the_week(day_of_the_week):
  acceptable_distance = 3
  if pylev.levenshtein('lunes', day_of_the_week) <= acceptable_distance:
    return 'lunes'
  elif pylev.levenshtein('martes', day_of_the_week) <= acceptable_distance:
    return 'martes'
  elif pylev.levenshtein('miércoles', day_of_the_week) <= acceptable_distance:
    return 'miércoles'
  elif pylev.levenshtein('jueves', day_of_the_week) <= acceptable_distance:
    return 'jueves'
  elif pylev.levenshtein('viernes', day_of_the_week) <= acceptable_distance:
    return 'viernes'
  elif pylev.levenshtein('sábado', day_of_the_week) <= acceptable_distance:
    return 'sábado'
  elif pylev.levenshtein('domingo', day_of_the_week) <= acceptable_distance:
    return 'domingo'
  else:
    return 'bad day'

# Aplico la función creada sobre la columna "DIA"
homicides_df['DIA'] = homicides_df['DIA'].apply(get_correct_day_of_the_week)

# Verifico cuales son los valores únicos en la columna "DIA"
homicides_df['DIA'].unique()

array(['lunes', 'jueves', 'viernes', 'sábado', 'domingo', 'martes',
       'miércoles'], dtype=object)

Ahora los días de la semana son los 7 correctos

# Corregir el año de nacimiento según la edad

In [84]:
homicides_df[['AÑO DE NACIMIENTO', 'EDAD']].head(10)

,AÑO DE NACIMIENTO,EDAD
1,1985.0,30.0
2,1982.0,33.0
3,1975.0,40.0
4,1949.0,66.0
5,1973.0,42.0
6,1990.0,25.0
7,1978.0,37.0
9,1985.0,30.0
10,1974.0,41.0
11,1985.0,30.0


Se puede ver que el año de nacimiento y la edad de la persona cuadran como si estuviéramos en el año 2015

<span style="color:red">TODO: Fix the "AÑO DE NACIMIENTO" column using the column "EDAD".</span>

In [85]:
from datetime import date

# Obtengo el año actual
current_year = date.today().year

# Aplico una función para calcular el año de nacimiento a partir del año actual
# y la edad de la persona
homicides_df['AÑO DE NACIMIENTO'] = homicides_df.apply(lambda x: current_year - x.EDAD, axis=1)

# Ahora el año de nacimiento ya está corrregido
homicides_df[['AÑO DE NACIMIENTO', 'EDAD']].head(10)

,AÑO DE NACIMIENTO,EDAD
1,1992.0,30.0
2,1989.0,33.0
3,1982.0,40.0
4,1956.0,66.0
5,1980.0,42.0
6,1997.0,25.0
7,1985.0,37.0
9,1992.0,30.0
10,1981.0,41.0
11,1992.0,30.0


Los años de nacimiento ya muestran el valor corregido